In [ ]:
%pip install qiskit
%pip install qiskit-aer
%pip install qiskit-ibm-runtime
%pip install qiskit-transpiler-service
%pip install colorama
%pip install matplotlib
%pip install pylatexenc

In [ ]:
# 사용할 함수 import
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.transpiler.preset_passmanagers.plugin import list_stage_plugins
from qiskit.circuit.library import QFT
from qkh2024.grader import scorer
from itertools import product
import pandas as pd

import matplotlib.pyplot as plt
import numpy as np

In [ ]:
service = QiskitRuntimeService()
backend = service.backend("ibm_sherbrooke")
scorer = scorer()

In [ ]:
layoutValue = list_stage_plugins("layout")
routingValue = list_stage_plugins("routing")
translationValue = list_stage_plugins("translation")
initValue = list_stage_plugins("init")

routingValue.pop(2)

print(
    "stage check" 
    "\n layout:", layoutValue,
    "\n routing:", routingValue, 
    "\n translation:", translationValue,
    "\n init:", initValue)

In [ ]:
qc = QFT(5)
initial_circuit = qc
first_output = True     # 첫번째 리스트 항목인지 확인
pm_best_score = 1       # 가장 좋은 score
pm_best_score_index = 0 # 해당 score가 몇 번째인지

# state 값 별 모든 경우의 수를 계산하여 score 계산
for i, (layout, routing, translation, init) in enumerate(product(layoutValue, routingValue, translationValue, initValue)):
    pm_best = generate_preset_pass_manager(
        optimization_level=0,
        backend=backend,
        layout_method=layout,
        routing_method=routing,
        translation_method=translation,
        init_method=init
    )

    transpiled_circuit = pm_best.run(initial_circuit)
    now_score = scorer.score(transpiled_circuit, backend)

    # 현재 i번째 상태 값과 계산 결과(score)를 now에 저장
    # pandas에서 제공하는 dataFrame에 추가
    now = [{'layout': layout, 'routing': routing, 'translation': translation, 'init': init, 'score': now_score}]
    df = pd.DataFrame(now, columns=['layout', 'routing', 'translation', 'init', 'score'], index=[i])

    # 리스트의 맨 윗 부분인 경우에만 위쪽에 header 달리도록
    if first_output:
        print(df)
        first_output = False
    else:
        print(df.to_string(header=False))

    
    # 저장된 best score보다 현재 score가 더 작은 경우 best score 변경
    # score가 위치한 인덱스 번호도 추가 => 보기 편하라고
    if (pm_best_score > now_score):
        pm_best_score = now_score
        pm_best_score_index = i

# 가장 좋은 score 값 출력
print("best index & score: ", pm_best_score_index,"번째 ", pm_best_score)